Implement the ring-plots from the [MinGenome](https://pubs.acs.org/doi/10.1021/acssynbio.7b00296).

[Bokeh "Burtin" example](https://bokeh.pydata.org/en/latest/docs/gallery/burtin.html) may provide some helpful hints.

Question: Encode the protein value on the area or height?  How is that read?

Next steps to include:
* Genes we **should** be knocking out at each step?
* Genes we think are essential at each step?
* color regions knocked out already?


In [ ]:
import pandas as pd
import numpy as np

from collections import OrderedDict
from math import log, sqrt

import bokeh
from bokeh.plotting import figure, curdoc
from bokeh.io import show, output_notebook, export_png

from bokeh.models import ColumnDataSource, CategoricalColorMapper, Whisker, LabelSet, Div
from bokeh.models import HoverTool, BoxSelectTool, PanTool, WheelZoomTool, ResetTool, SaveTool, ColorBar
from bokeh.models import widgets
from bokeh.models import formatters 
from bokeh.models.ranges import FactorRange
from bokeh.models.formatters import PrintfTickFormatter, FuncTickFormatter

from bokeh.transform import factor_cmap, linear_cmap, dodge

from bokeh import events
from bokeh.layouts import column, row, widgetbox, layout, gridplot
from bokeh import palettes

from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid
from bokeh.models.glyphs import HBar
from bokeh.transform import factor_cmap
import bokeh.palettes as palettes
from bokeh.palettes import PiYG, Spectral6, Category10, Category20_20

import colorcet

output_notebook()
%matplotlib inline

In [ ]:
bokeh.__version__
#TODO: When version is 1.0.4 then try bokeh/latex: https://bokeh.pydata.org/en/latest/docs/user_guide/extensions_gallery/latex.html

In [ ]:
from math import ceil 

def square_shape(N, sh=1, sw=1):
    #https://stackoverflow.com/questions/339939/stacking-rectangles-to-into-the-most-square-like-arrangement-possible
    cols = round(sqrt(N * sh / sw))
    rows = ceil(N / cols)
    return rows, cols

In [ ]:
knockouts = pd.read_csv("./Blattner_ecoli_kos.tab",sep='\t')
knockouts.columns.values[0] = "locus"
knockouts = knockouts.set_index("locus")
knockouts.head()

In [ ]:
knockouts[knockouts['Step'] <=10].to_csv('Blattner_ecoli_kos_1-10.tab',sep='\t')

In [ ]:
observed = pd.read_csv("E.coli_avg_iBAQ_mass_pct_induced_v_uninduced.tab", sep="\t").set_index("Blattner").drop("index", axis="columns")
observed.head()

In [ ]:
predicted = pd.read_csv("./protein_mass_percent_predicted_from_previous_step.csv").set_index("level_0")
predicted.tail()

In [ ]:
genes = pd.read_csv("./E_coli_metadata.txt", sep="\t")[["gene", "locus", "start", "stop"]]
genes = genes[~genes["locus"].str.startswith("unique")].reset_index(drop=True)
genes = genes.set_index("locus").sort_values("start")
genes = genes[~genes.index.duplicated(keep="first")]
genes = genes.assign(center=(genes["stop"]-genes["start"])/2+ genes["start"],
                     seq=np.arange(genes.shape[0]))

max_loc = genes.stop.max()
genes = genes.assign(start_pct = genes.start/max_loc,
                     stop_pct = genes.stop/max_loc,
                     center_pct = genes.center/max_loc)

genes = genes.assign(stop_rad = -np.radians(genes.start_pct*360-90),
                     start_rad = -np.radians(genes.stop_pct*360-90),
                     center_rad = -np.radians(genes.center_pct*360-90))

genes = genes.drop(["start_pct", "stop_pct", "center_pct"], axis="columns")
genes.head()

In [ ]:
import re

def enlarge(json, factor=4, indent=None):
    """Enlarge things by the given factor.
    Will only enlarge explicilty set, since all other values are determined by the theme.
    TODO: Implement an enlarged theme as well!
    """
    
    def mul(v):
        if v is None: return v

        try: 
            if v.endswith("pt") or v.endswith("em"): 
                size = int(v[:-2])*factor
                return f"{size}{v[-2:]}"
        except: pass

        try:
            c = v.copy()
            c["value"] = mul(c["value"])
            return c
        except: pass

        try:  
            return v*factor
        except: pass


        raise ValueError(f"Cannot Enlarge {v}")
    
    patterns = ["^plot_width$", "^plot_height$", "^.*font_size$", "^.*_standoff$"]
    patterns = [re.compile(p) for p in patterns]
    
    def _indent():
        return f"{indent}{indent[0]}" if indent else None
    
    if type(json) is list:
        l = []
        for i in range(len(json)):
            if indent: print(f"l: {indent}{i}")
            l.append(enlarge(json[i], factor, _indent()))
        return l
    elif type(json) is dict:
        d = {}
        for k,v in json.items():
            if indent: print(f"d: {indent}{k}")
            matched = any([p.match(k) for p in patterns])
            if matched: 
                d[k] = mul(v)
            else: 
                d[k] = enlarge(json[k], factor, _indent())

        return d
    else:
        return json

In [ ]:
def export(p, filename, factor=4):
    "Rescale components & fonts. Save to file"
    
    big_theme = {
        'attrs' : {
            'Title': {
                'text_font_size': f'{10*factor}pt'
            },
            'ColorBar':{
                'title_text_font_size': f'{8*factor}pt',
                'major_label_text_font_size': f'{8*factor}pt',
                'major_label_text_align': "left"
            },
            'Axis': {
                'major_label_text_font_size': f'{8*factor}pt',
                'axis_label_text_font_size': f'{8*factor}pt'
            }

        }
    }
    theme = bokeh.themes.Theme(json=big_theme)

    jdoc = p.document.to_json()
    jdoc2 = enlarge(jdoc, factor)
    doc2 = bokeh.document.Document.from_json(jdoc2)
    doc2.theme = theme
    bokeh.io.reset_output()
    output_notebook()

    p2 = doc2.get_model_by_id(p.id)
    p2.toolbar.logo = None
    p2.toolbar_location = None
        
    bokeh.io.export_png(p2, filename, width=p.plot_width*factor, height=p.plot_height*factor)
    return p2

# Gene Block

In [ ]:
#Isolate data
gene_drop = genes.join(knockouts.drop(['gene', 'E_coli_W3110'], axis="columns") , how="left")\
                 .drop(["stop_rad", "start_rad", "center_rad", "center"], axis="columns")
rows, cols = square_shape(gene_drop.seq.max())
gene_drop = gene_drop.assign(col = gene_drop.seq%cols, row=-(gene_drop.seq//rows),
                             start = gene_drop.start/1000,
                             stop = gene_drop.stop/1000)

#gene_drop.head()

In [ ]:
# Prep image
#rows, cols = square_shape(end)
img = np.full((rows, cols), np.nan).ravel()

for start, stop, v in gene_drop[["start", "stop", "Step"]].values:
    img[int(start):int(stop)] =v

img = np.flipud(img.reshape((rows, cols)))

In [ ]:
#Visualization
def block_diagram(interactive=False):
    bokeh.io.curdoc().clear()
    image = np.flipud(img)
    round_step_top = ((gene_drop.Step.max()//10)+1)*10  #TODO: Should I use this rounded-up number or just max?  
    cmap = linear_cmap('Step', palettes.Viridis256 , low=0, high=round_step_top, 
                       nan_color='#efefef')["transform"]

    row_scale = cols
    col_scale = rows
    
    #There are problems with combining: images, tool-tips and inverted Y
    # There are also problems with export_png when using a FuncTickFormatter...
    # This block of configuration lets you do either with just a flag.
    if interactive:
        tip = [("", "@image")] if interactive else None
        y_range = (-rows*row_scale,0)
        y = -rows*row_scale
        y_formatter = FuncTickFormatter(code="return `${tick*-1} kbp`")
    else:
        tip = None
        y_range=(rows*row_scale, 0)
        y=rows*row_scale
        y_formatter = PrintfTickFormatter(format="%d kbp")
    
    
    p = figure(width=600, height=550, x_range=(0,cols*col_scale), y_range=y_range,
               title="Gene Deletions: Step & Location",
               tooltips=tip)

    p.yaxis.formatter = y_formatter
    p.xaxis.visible=False
    p.image(image=[image], x=0, y=y, dw=cols*col_scale, dh=rows*row_scale, color_mapper=cmap)

    colorbar = ColorBar(color_mapper=cmap, location=(0,0), title="Step", name="colorbar")
    p.add_layout(colorbar, "right")
    return p

p = block_diagram(interactive=False)
show(p)
export(p, "gene_block.png")

# Protein Rings

In [ ]:
def ring_plot(data, rng, cmap, *, title="", out_steps=3):
    bokeh.io.curdoc().clear()

    size = 400
    inner_radius = size/4
    
    if "outer" not in data.columns: data = data.assign(outer=size/100)
    if "inner" not in data.columns: data = data.assign(inner=0)
     
    data = data.assign(inner_radius=data.inner+inner_radius)\
               .assign(outer_radius=data.outer+inner_radius)
        
    source = ColumnDataSource(data)

    span = data.outer_radius.abs().max()*1.1

    p = figure(plot_width=size, plot_height=size,
                x_range=(span, -span), y_range=(span,-span),
                title=title)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    p.annular_wedge(source=source,
                    x=0,y=0, 
                    inner_radius="inner_radius", outer_radius="outer_radius",
                    start_angle="start_rad", end_angle="stop_rad",
                    color=cmap)
    p.title.text_font_size="10pt"

    
    # Radial labels
    #TODO: Need to "nice" these steps.  Right now they are very rough numbers.
    out_radii = np.linspace(inner_radius, data.outer_radius.max(), out_steps)
    step_size = out_radii[1] - out_radii[0]
    in_radii = np.flip(np.arange(inner_radius, 0, -step_size)[1:])
    radii = np.concatenate([in_radii, out_radii])
    
    out_labels = np.linspace(0, rng[1], out_radii.shape[0])
    in_labels = np.linspace(-in_radii.shape[0]*out_labels[1], 0-out_labels[1], in_radii.shape[0])
    labels = np.concatenate([in_labels, out_labels])

    #Only show one more inner circle than values (Strictly speaking)
    cutoff = np.where(labels < rng[0])[0]
    cutoff= cutoff[-1] if cutoff.shape[0] > 0 else 0
    
    p.circle(0, 0, radius=radii[cutoff:], fill_color=None, line_color="lightgray")
    p.text(0, radii[cutoff:], [f"{r:9.2f}" for r in labels[cutoff:]],
           text_font_size="8pt", text_align="center", text_baseline="middle")

    return p

In [ ]:
data = genes.assign(ab = (genes.seq%2).apply(str))\
            .assign(outer=np.linspace(1,10, genes.shape[0]))
    
cmap = factor_cmap('ab', ["#F05974", "#260D75"], ["0", "1"])

show(ring_plot(data, (0, 1), cmap, title="A/B Seq Map"))

In [ ]:
bokeh.io.export_png(ring_plot(data, (0,1), cmap, title="A/B Seq Map"))

In [ ]:
observed.columns

In [ ]:
from bokeh.layouts import gridplot
def grid(plots, size=150):
    rows, cols = square_shape(len(plots))
    return gridplot(plots, ncols=cols, plot_width=size, plot_height=size)


In [ ]:
def single_condition(condition, *, cmap=None, cdata=None, base=None, scale=10000):
    vs = condition if base is None else base**condition
    vs = vs/vs.sum()*scale

    data = genes.join(vs, how="left")\
                .fillna(0)\
                .rename({condition.name:"outer"}, axis="columns")

    if cmap is None or cdata is None:
        data = data.assign(ab = (genes.seq%2).apply(str))
        cmap = factor_cmap('ab', ["#F05974", "#260D75"], ["0", "1"])
    else:
        data = data.join(cdata, how="left")

    plot =  ring_plot(data, (condition.min(), condition.max()), cmap)
    plot.title.text=condition.name
    return plot

show(single_condition(observed["W3110_control"], base=2))


In [ ]:
def delta_plot(focus, reference, genes, cmap=["#E02E4E", "#4D31A5"], *, scale=500):
    """
    focus -- Values of the "current" step.  Presents focus-reference.
    reference -- Values of the step current is being compared to
    genes -- Layout information
    cmap -- A pair of colors, (Positive-color, Negative-color)
    scale -- A rescale-factor to "zoom" the plot...sort of
    """
    delta = (focus - reference).rename("delta")*scale
    data = genes.join(delta, how="left")\
                .fillna(0)
    data = data.assign(pn = data["delta"].apply(lambda v: "+" if v > 0 else "-"))\
                .rename({"delta": "outer"}, axis="columns")

    cmap = factor_cmap('pn', cmap, ["+", "-"])

    plot =  ring_plot(data, (data["outer"].min()/scale, data["outer"].max()/scale),
                                 cmap, title = f"{focus.name} vs {reference.name}")
    
    return plot

In [ ]:
show(delta_plot(observed["Step_06_control"], observed["W3110_control"], genes))

In [ ]:
g = grid([single_condition(observed[condition]) for condition in observed.columns])
show(g)

In [ ]:
show(single_condition(predicted[predicted.columns[-6]], base=2))


In [ ]:
step_integers = (655104, 65535, 590079, 16711924, 16711700,)
def get_RGB_from_I(RGBint):
    blue =  RGBint & 255
    green = (RGBint >> 8) & 255
    red =   (RGBint >> 16) & 255
    return red, green, blue

def get_I_from_RGB(rgb):
    red = rgb[0]
    green = rgb[1]
    blue = rgb[2]
    print(red, green, blue)
    RGBint = (red<<16) + (green<<8) + blue
    return RGBint

def get_hex_from_RGB( rgb ):
    return 
    
def get_hex_from_I( RGBint ):
    r,g,b = get_RGB_from_I( RGBint )
    return "#{0:02x}{1:02x}{2:02x}".format(r,g,b)
step_hex = [get_hex_from_I(step) for step in step_integers]
#step_color = {"Step05": "#00ffff", "Step09": '#0900ff', "Step10": '#ff00f4', "W3110":'#ff0014' }
step_color = dict(zip(["Step10", "Step09", "W3110","Step05"], 
                      bokeh.palettes.Colorblind[4]))

In [ ]:
step_color

In [ ]:
step_predictions = """Step05_predicted_from_W3110
Step09_predicted_from_Step05
Step10_predicted_from_W3110
Step10_predicted_from_Step09""".split('\n')
plots = []
for step_predicted in step_predictions:
    step_measured = 'iBAQ_{}'.format(step_predicted[:6])
    positive_mask = (predicted[step_predicted].apply(np.log2) - predicted[step_measured].apply(np.log2)) > 0
    negative_mask = ~positive_mask
    over_predicted = predicted[positive_mask][step_predicted].sum()
    under_predicted = predicted[negative_mask][step_predicted].sum()
    net_predicted = over_predicted - under_predicted
    kl_div = predicted[r'$D_{KL}\left(\text{%s}\|\text{%s}\right)$' % (step_measured, step_predicted)].sum()
    print('Overpredicted: {:.0%}\nUnderpredicted: {:.0%}\nNet predicted: {:.0%}\nKL-divergence: {}'.format(over_predicted,
                                                                             under_predicted,
                                                                             net_predicted, 
                                                                                                          kl_div))
    cmap = (step_color[step_predicted.split("_")[0]],
            step_color[step_predicted.split("_")[-1]])
    p = delta_plot(predicted[step_predicted].apply(np.log2).fillna(0), 
                predicted[step_measured].apply(np.log2).fillna(0),
                genes, cmap, scale=5 )
    plots.append(p)
    show(p)
    export(p, '{}_vs_{}.png'.format( step_predicted, 
                                     step_measured ))
# grid_plot = grid(plots)
# show(grid_plot)
#export(grid_plot, 'grid_plot.png')

In [ ]:
from IPython.display import Latex, HTML
Latex(predicted.columns[-6])

In [ ]:
for i in range(6):
    display(Latex(predicted.columns[-6+i]))

In [ ]:
print('\n'.join(predicted.columns))

In [ ]:
novel_predictions = pd.read_csv("genome_view.tab", sep="\t")
novel_predictions = novel_predictions[novel_predictions["$gene_or_promoter"].str.startswith("b")]\
                        .rename({"$gene_or_promoter": "gene_or_promoter",
                                 "mass reclaimed $(fg/cell)$": "mass reclaimed (fg/cell)"}, axis="columns")\
                        .set_index("gene_or_promoter")

novel_predictions
sorted(set(novel_predictions['mass reclaimed (fg/cell)'].values))

In [ ]:
# [<2 fg/cell,  <4 fg/cell, < 8 fg/cell, < 16 fg/cell, < 32 fg/cell]
magnitudes = novel_predictions["mass reclaimed (fg/cell)"].apply(np.log2).apply(ceil).apply(str)
magnitudes.name = "magnitude"
factors = magnitudes.unique()
cmap =  factor_cmap('magnitude', palettes.viridis(len(factors)), factors = factors)

In [ ]:
p= single_condition(novel_predictions["mass reclaimed (fg/cell)"], cmap=cmap, cdata=magnitudes)
show(p)
export(p, "reclaimed.png")